In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from scipy.stats import mode

In [2]:
previous_application = pd.read_csv("../data/raw/previous_application.csv")
application_train = pd.read_csv("../data/raw/application_train.csv")
bureau = pd.read_csv("../data/raw/bureau.csv")

In [3]:
time_periods = [3, 6, 12, 24, 36]
aggregators = ['sum', 'mean', 'max', 'min', 'std']

In [4]:
def vin_generic_aggregator(df, groupby_column, agg_column, agg_funcs):
    aggregated_df = df.groupby(groupby_column).agg({agg_column: agg_funcs}).reset_index()
    aggregated_df.columns = ['_'.join(col).rstrip('_') for col in aggregated_df.columns.values]
    return aggregated_df

In [5]:
# 1. Vintage - Months Since Last Approved Loan (based on DAYS_DECISION)
def vin_months_since_last_approved(previous_application, agg_funcs=['max']):
    last_approved = previous_application[previous_application['NAME_CONTRACT_STATUS'] == 'Approved']
    last_approved = vin_generic_aggregator(last_approved, 'SK_ID_CURR', 'DAYS_DECISION', agg_funcs)
    for agg_func in agg_funcs:
        last_approved[f'vin_months_since_last_approved_{agg_func}'] = last_approved[f'DAYS_DECISION_{agg_func}'] / -30
    return last_approved[['SK_ID_CURR'] + [f'vin_months_since_last_approved_{agg_func}' for agg_func in agg_funcs]]
vin_months_since_last_approved(previous_application)

,SK_ID_CURR,vin_months_since_last_approved_max
0,100001,58.000000
1,100002,20.200000
2,100003,24.866667
3,100004,27.166667
4,100005,25.233333
...,...,...
337693,456251,9.100000
337694,456252,83.233333
337695,456253,63.633333
337696,456254,9.233333


In [6]:
# 2. Vintage - Days Since Last Loan Rejection (based on DAYS_DECISION)
def vin_days_since_last_rejection(previous_application, agg_funcs=['max']):
    last_rejected = previous_application[previous_application['NAME_CONTRACT_STATUS'] == 'Refused']
    last_rejected = vin_generic_aggregator(last_rejected, 'SK_ID_CURR', 'DAYS_DECISION', agg_funcs)
    for agg_func in agg_funcs:
        last_rejected[f'days_since_last_rejection_{agg_func}'] = last_rejected[f'DAYS_DECISION_{agg_func}'] * -1
    return last_rejected[['SK_ID_CURR'] + [f'days_since_last_rejection_{agg_func}' for agg_func in agg_funcs]]

vin_days_since_last_rejection(previous_application, aggregators)


,SK_ID_CURR,days_since_last_rejection_sum,days_since_last_rejection_mean,days_since_last_rejection_max,days_since_last_rejection_min,days_since_last_rejection_std
0,100006,181,181.000,181,181,NaN
1,100011,1162,1162.000,1162,1162,NaN
2,100027,181,181.000,181,181,NaN
3,100030,20539,2053.900,840,2689,-656.525272
4,100035,1147,143.375,119,160,-15.738374
...,...,...,...,...,...,...
118272,456244,1592,1592.000,1592,1592,NaN
118273,456247,650,650.000,650,650,NaN
118274,456249,1454,1454.000,1454,1454,NaN
118275,456250,454,454.000,454,454,NaN


In [7]:
# 3. Vintage - Number of Loans Approved in the Last N Months (based on DAYS_DECISION)
def vin_num_loans_approved_last_n_months(previous_application, months=12):
    recent_approved = previous_application[(previous_application['NAME_CONTRACT_STATUS'] == 'Approved') &
                                           (previous_application['DAYS_DECISION'] >= -30 * months)]
    num_approved = recent_approved.groupby('SK_ID_CURR').size().reset_index(name=f'vin_num_loans_approved_last_{months}_months')
    return num_approved

vin_num_loans_approved_last_n_months(previous_application)

,SK_ID_CURR,vin_num_loans_approved_last_12_months
0,100006,3
1,100009,3
2,100013,1
3,100014,1
4,100018,2
...,...,...
138457,456245,1
138458,456246,1
138459,456250,1
138460,456251,1


In [8]:
# 4. Vintage - Days Since Last Payment Default (based on DAYS_CREDIT)
def vin_days_since_last_default(bureau, agg_funcs=['min']):
    defaulted = bureau[bureau['CREDIT_DAY_OVERDUE'] > 0]
    last_default = vin_generic_aggregator(defaulted, 'SK_ID_CURR', 'DAYS_CREDIT', agg_funcs)
    for agg_func in agg_funcs:
        last_default[f'days_since_last_default_{agg_func}'] = last_default[f'DAYS_CREDIT_{agg_func}'] * -1
    return last_default[['SK_ID_CURR'] + [f'days_since_last_default_{agg_func}' for agg_func in agg_funcs]]

vin_days_since_last_default(bureau)

,SK_ID_CURR,days_since_last_default_min
0,100120,610
1,100162,1998
2,100341,333
3,100349,301
4,100472,197
...,...,...
3859,455858,1245
3860,455865,742
3861,455988,349
3862,456112,93


In [9]:
# 5. Vintage - Number of Previous Applications Over Time (based on DAYS_DECISION)
def vin_num_previous_applications_last_n_months(previous_application, months=12):
    recent_apps = previous_application[previous_application['DAYS_DECISION'] >= -30 * months]
    num_apps = recent_apps.groupby('SK_ID_CURR').size().reset_index(name=f'num_previous_applications_last_{months}_months')
    return num_apps

vin_num_previous_applications_last_n_months(previous_application)

,SK_ID_CURR,num_previous_applications_last_12_months
0,100005,1
1,100006,7
2,100008,1
3,100009,3
4,100012,1
...,...,...
193493,456246,1
193494,456250,1
193495,456251,1
193496,456254,2


In [10]:
# 6. Vintage - Last Loan Taken by Category (based on DAYS_DECISION)
def vin_last_loan_taken_by_category(previous_application, agg_funcs=['max']):
    last_loan = previous_application.groupby(['SK_ID_CURR', 'NAME_CONTRACT_TYPE'])['DAYS_DECISION'].agg(agg_funcs).reset_index()
    for agg_func in agg_funcs:
        last_loan[f'vin_days_since_last_loan_{agg_func}'] = last_loan[agg_func] * -1
    last_loan_pivot = last_loan.pivot(index='SK_ID_CURR', columns='NAME_CONTRACT_TYPE', values=[f'vin_days_since_last_loan_{agg_func}' for agg_func in agg_funcs]).reset_index()
    last_loan_pivot.columns = ['SK_ID_CURR'] + [f'vin_last_loan_{col[1].lower()}_{col[0].split("_")[-2]}' for col in last_loan_pivot.columns[1:]]
    return last_loan_pivot


vin_last_loan_taken_by_category(previous_application)


,SK_ID_CURR,vin_last_loan_cash loans_loan,vin_last_loan_consumer loans_loan,vin_last_loan_revolving loans_loan,vin_last_loan_xna_loan
0,100001,NaN,1740.0,NaN,NaN
1,100002,NaN,606.0,NaN,NaN
2,100003,746.0,828.0,NaN,NaN
3,100004,NaN,815.0,NaN,NaN
4,100005,315.0,757.0,NaN,NaN
...,...,...,...,...,...
338852,456251,NaN,273.0,NaN,NaN
338853,456252,NaN,2497.0,NaN,NaN
338854,456253,NaN,1909.0,NaN,NaN
338855,456254,NaN,277.0,NaN,NaN


In [11]:
# 7. Vintage - Days Since Last Loan Closure (based on DAYS_ENDDATE_FACT)
def vin_days_since_last_loan_closure(bureau, agg_funcs=['max']):
    closed_loans = bureau[bureau['CREDIT_ACTIVE'] == 'Closed']
    last_closure = vin_generic_aggregator(closed_loans, 'SK_ID_CURR', 'DAYS_ENDDATE_FACT', agg_funcs)
    for agg_func in agg_funcs:
        last_closure[f'vin_days_since_last_loan_closure_{agg_func}'] = last_closure[f'DAYS_ENDDATE_FACT_{agg_func}'] * -1
    return last_closure[['SK_ID_CURR'] + [f'vin_days_since_last_loan_closure_{agg_func}' for agg_func in agg_funcs]]

vin_days_since_last_loan_closure(bureau, aggregators)


,SK_ID_CURR,vin_days_since_last_loan_closure_sum,vin_days_since_last_loan_closure_mean,vin_days_since_last_loan_closure_max,vin_days_since_last_loan_closure_min,vin_days_since_last_loan_closure_std
0,100001,3302.0,825.500000,544.0,1328.0,-369.078582
1,100002,4185.0,697.500000,36.0,1185.0,-515.992539
2,100003,3292.0,1097.333333,540.0,2131.0,-896.097279
3,100004,1065.0,532.500000,382.0,683.0,-212.839141
4,100005,123.0,123.000000,123.0,123.0,NaN
...,...,...,...,...,...,...
267920,456249,15059.0,1369.000000,291.0,2525.0,-665.667034
267921,456250,760.0,760.000000,760.0,760.0,NaN
267922,456253,1588.0,794.000000,794.0,794.0,-0.000000
267923,456254,859.0,859.000000,859.0,859.0,NaN


In [12]:
# 8. Vintage - Days Since First Loan Taken (based on DAYS_CREDIT)
def vin_days_since_first_loan_taken(bureau, agg_funcs=['max']):
    first_loan = vin_generic_aggregator(bureau, 'SK_ID_CURR', 'DAYS_CREDIT', agg_funcs)
    for agg_func in agg_funcs:
        first_loan[f'vin_days_since_first_loan_taken_{agg_func}'] = first_loan[f'DAYS_CREDIT_{agg_func}'] * -1
    return first_loan[['SK_ID_CURR'] + [f'vin_days_since_first_loan_taken_{agg_func}' for agg_func in agg_funcs]]
vin_days_since_first_loan_taken(bureau, aggregators)

,SK_ID_CURR,vin_days_since_first_loan_taken_sum,vin_days_since_first_loan_taken_mean,vin_days_since_first_loan_taken_max,vin_days_since_first_loan_taken_min,vin_days_since_first_loan_taken_std
0,100001,5145,735.000000,49,1572,-489.942514
1,100002,6992,874.000000,103,1437,-431.451040
2,100003,5603,1400.750000,606,2586,-909.826128
3,100004,1734,867.000000,408,1326,-649.124025
4,100005,572,190.666667,62,373,-162.297053
...,...,...,...,...,...,...
305806,456249,21672,1667.076923,483,2713,-638.202353
305807,456250,2586,862.000000,760,1002,-125.395375
305808,456253,3470,867.500000,713,919,-103.000000
305809,456254,1104,1104.000000,1104,1104,NaN


In [13]:
# 9. Vintage - Number of Times Delinquent in Last N Months (based on DAYS_CREDIT)
def vin_num_times_delinquent_last_n_months(bureau, months=6):
    recent_delinquencies = bureau[(bureau['CREDIT_DAY_OVERDUE'] > 0) &
                                  (bureau['DAYS_CREDIT'] >= -30 * months)]
    num_delinquent = recent_delinquencies.groupby('SK_ID_CURR').size().reset_index(name=f'vin_num_times_delinquent_last_{months}_months')
    return num_delinquent

vin_num_times_delinquent_last_n_months(bureau)

,SK_ID_CURR,vin_num_times_delinquent_last_6_months
0,101033,1
1,101300,1
2,101391,1
3,102835,1
4,103845,1
...,...,...
456,452063,1
457,454719,1
458,454815,1
459,455189,1


In [14]:
# 10. Vintage - Number of Overdue Loans in Last N Months (based on DAYS_CREDIT)
def vin_num_overdue_loans_last_n_months(bureau, months=6):
    recent_overdue = bureau[(bureau['AMT_CREDIT_SUM_OVERDUE'] > 0) &
                            (bureau['DAYS_CREDIT'] >= -30 * months)]
    num_overdue = recent_overdue.groupby('SK_ID_CURR').size().reset_index(name=f'vin_num_overdue_loans_last_{months}_months')
    return num_overdue
vin_num_overdue_loans_last_n_months(bureau)

,SK_ID_CURR,vin_num_overdue_loans_last_6_months
0,101033,1
1,101281,1
2,101300,1
3,101391,1
4,102835,1
...,...,...
493,452063,1
494,454719,1
495,454815,1
496,455189,1


In [15]:
# # 11. Vintage - Aggregates of Credit Amounts (based on AMT_CREDIT_SUM)
# def vin_aggregated_credit_amounts(bureau, agg_funcs=aggregators):
#     aggregated_credit = vin_generic_aggregator(bureau, 'SK_ID_CURR', 'AMT_CREDIT_SUM', agg_funcs)
#     aggregated_credit.columns = [f'vin_credit_amount_{agg_func}' if col != 'SK_ID_CURR' else col for col, agg_func in zip(aggregated_credit.columns, [''] + agg_funcs)]
#     return aggregated_credit
# vin_aggregated_credit_amounts(bureau)

In [16]:
# # List of functions to calculate vintage variables
# vintage_functions = [
#     vin_months_since_last_approved,
#     vin_days_since_last_rejection,
#     vin_days_since_last_default,
#     vin_last_loan_taken_by_category,
#     vin_days_since_last_loan_closure,
#     vin_days_since_first_loan_taken,
#     vin_aggregated_credit_amounts
# ]

In [17]:
# resultant_df = application_train[['SK_ID_CURR']]

In [18]:
# for months in time_periods:
#     time_based_functions = [
#         vin_num_loans_approved_last_n_months,
#         vin_num_previous_applications_last_n_months,
#         vin_num_times_delinquent_last_n_months,
#         vin_num_overdue_loans_last_n_months
#     ]
#     for func in time_based_functions:
#         feature_df = func(previous_application if 'previous_application' in func.__code__.co_varnames else bureau, months=months)
#         resultant_df = resultant_df.merge(feature_df, on='SK_ID_CURR', how='left')

In [19]:
# for func in vintage_functions:
#     feature_df = func(previous_application if 'previous_application' in func.__code__.co_varnames else bureau, agg_funcs=aggregators)
#     resultant_df = resultant_df.merge(feature_df, on='SK_ID_CURR', how='left')

In [20]:
# fill_rate = resultant_df.notna().mean().reset_index()
# fill_rate.columns = ['Feature', 'FillRate']
# print(fill_rate)

In [21]:
# resultant_df.to_csv('/content/drive/MyDrive/UMD/DATA602_DATA_Science/Project/home-credit-default-risk/cleaned_vintage_features.csv', index=False)

In [22]:
# 12. Vintage - Average Time Between Loans (based on DAYS_DECISION)
def vin_avg_time_between_loans(previous_application):
    approved_loans = previous_application[previous_application['NAME_CONTRACT_STATUS'] == 'Approved']
    approved_loans = approved_loans.sort_values(by=['SK_ID_CURR', 'DAYS_DECISION'])
    approved_loans['time_diff'] = approved_loans.groupby('SK_ID_CURR')['DAYS_DECISION'].diff().abs()
    avg_time_between = approved_loans.groupby('SK_ID_CURR')['time_diff'].mean().reset_index(name='vin_avg_time_between_loans')
    return avg_time_between
vin_avg_time_between_loans(previous_application)

,SK_ID_CURR,vin_avg_time_between_loans
0,100001,NaN
1,100002,NaN
2,100003,797.5
3,100004,NaN
4,100005,NaN
...,...,...
337693,456251,NaN
337694,456252,NaN
337695,456253,942.0
337696,456254,45.0


In [23]:
# # 13. Vintage - Ratio of Approved to Rejected Loans (based on NAME_CONTRACT_STATUS)
# def vin_ratio_approved_to_rejected(previous_application):
#     approved = previous_application[previous_application['NAME_CONTRACT_STATUS'] == 'Approved'].groupby('SK_ID_CURR').size()
#     rejected = previous_application[previous_application['NAME_CONTRACT_STATUS'] == 'Refused'].groupby('SK_ID_CURR').size()
#     ratio = (approved / rejected).reset_index(name='vin_ratio_approved_to_rejected').fillna(0)
#     return ratio
# vin_ratio_approved_to_rejected(previous_application)

In [24]:
# # 14. Vintage - Maximum Credit Limit Over Time (based on AMT_CREDIT_SUM_LIMIT)
# def vin_max_credit_limit(bureau):
#     max_limit = bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_LIMIT'].max().reset_index(name='vin_max_credit_limit')
#     return max_limit
# vin_max_credit_limit(bureau)

In [25]:
# 15. Vintage - Number of Active Loans (based on CREDIT_ACTIVE)
def vin_num_active_loans(bureau):
    active_loans = bureau[bureau['CREDIT_ACTIVE'] == 'Active'].groupby('SK_ID_CURR').size().reset_index(name='vin_num_active_loans')
    return active_loans
vin_num_active_loans(bureau)


,SK_ID_CURR,vin_num_active_loans
0,100001,3
1,100002,2
2,100003,1
3,100005,2
4,100008,1
...,...,...
251810,456247,3
251811,456249,2
251812,456250,2
251813,456253,2


In [26]:
def vin_loan_prolongation_frequency(bureau):
    prolongation_frequency = bureau.groupby('SK_ID_CURR')['CNT_CREDIT_PROLONG'].sum().reset_index(name='vin_loan_prolongation_frequency')
    return prolongation_frequency

vin_loan_prolongation_frequency(bureau)

,SK_ID_CURR,vin_loan_prolongation_frequency
0,100001,0
1,100002,0
2,100003,0
3,100004,0
4,100005,0
...,...,...
305806,456249,0
305807,456250,0
305808,456253,0
305809,456254,0


In [27]:
# 17. Vintage - Total Overdue Amount in Last N Months (based on AMT_CREDIT_SUM_OVERDUE)
def vin_total_overdue_amount_last_n_months(bureau, months=6):
    recent_overdue = bureau[(bureau['DAYS_CREDIT'] >= -30 * months)]
    total_overdue = recent_overdue.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_OVERDUE'].sum().reset_index(name=f'vin_total_overdue_amount_last_{months}_months')
    return total_overdue
vin_total_overdue_amount_last_n_months(bureau)

,SK_ID_CURR,vin_total_overdue_amount_last_6_months
0,100001,0.0
1,100002,0.0
2,100005,0.0
3,100008,0.0
4,100016,0.0
...,...,...
93872,456226,0.0
93873,456231,0.0
93874,456234,0.0
93875,456243,0.0


In [28]:
# # 18. Vintage - Average Installment Payment as a Percentage of Income (based on AMT_ANNUITY and AMT_INCOME_TOTAL)
# def vin_avg_installment_income_ratio(application_train):
#     application_train['vin_avg_installment_income_ratio'] = application_train['AMT_ANNUITY'] / (application_train['AMT_INCOME_TOTAL'] + 1)
#     return application_train[['SK_ID_CURR', 'vin_avg_installment_income_ratio']]

# vin_avg_installment_income_ratio(application_train)

In [29]:
# 19. Vintage - Recency of Credit Update (based on DAYS_CREDIT_UPDATE)
def vin_days_since_credit_update(bureau, agg_funcs=['min']):
    credit_update = vin_generic_aggregator(bureau, 'SK_ID_CURR', 'DAYS_CREDIT_UPDATE', agg_funcs)
    for agg_func in agg_funcs:
        credit_update[f'vin_days_since_credit_update_{agg_func}'] = credit_update[f'DAYS_CREDIT_UPDATE_{agg_func}'] * -1
    return credit_update[['SK_ID_CURR'] + [f'vin_days_since_credit_update_{agg_func}' for agg_func in agg_funcs]]

vin_days_since_credit_update(bureau)

,SK_ID_CURR,vin_days_since_credit_update_min
0,100001,155
1,100002,1185
2,100003,2131
3,100004,682
4,100005,121
...,...,...
305806,456249,2498
305807,456250,127
305808,456253,701
305809,456254,401


In [30]:
# 20. Vintage - Number of Times Credit Limit Was Reached (based on AMT_CREDIT_SUM and AMT_CREDIT_SUM_LIMIT)
def vin_num_times_credit_limit_reached(bureau):
    limit_reached = bureau[bureau['AMT_CREDIT_SUM'] >= bureau['AMT_CREDIT_SUM_LIMIT']].groupby('SK_ID_CURR').size().reset_index(name='vin_num_times_credit_limit_reached')
    return limit_reached
vin_num_times_credit_limit_reached(bureau)

,SK_ID_CURR,vin_num_times_credit_limit_reached
0,100001,6
1,100002,4
2,100003,4
3,100004,2
4,100005,3
...,...,...
280341,456247,6
280342,456249,7
280343,456250,3
280344,456253,4


In [31]:
# # 21. Vintage - Credit Utilization Ratio (based on AMT_CREDIT_SUM and AMT_CREDIT_SUM_LIMIT)
# def vin_credit_utilization_ratio(bureau, agg_funcs=aggregators):
#     bureau['credit_utilization'] = bureau['AMT_CREDIT_SUM'] / bureau['AMT_CREDIT_SUM_LIMIT']
#     utilization = vin_generic_aggregator(bureau, 'SK_ID_CURR', 'credit_utilization', agg_funcs)
#     utilization.columns = [f'vin_credit_utilization_{agg_func}' if col != 'SK_ID_CURR' else col for col, agg_func in zip(utilization.columns, [''] + agg_funcs)]
#     return utilization
# vin_credit_utilization_ratio(bureau)

In [32]:
# 22. Vintage - Average Number of Rejections Over Time (based on DAYS_DECISION)
def vin_avg_rejections_last_n_months(previous_application, months=12):
    results = []
    # for months in time_periods:
    rejections = previous_application[(previous_application['NAME_CONTRACT_STATUS'] == 'Refused') &
                                      (previous_application['DAYS_DECISION'] >= -30 * months)]
    avg_rejections = rejections.groupby('SK_ID_CURR').size().reset_index(name=f'vin_avg_rejections_last_{months}_months')
    results.append(avg_rejections)
    return pd.concat(results, axis=1)
vin_avg_rejections_last_n_months(previous_application)

,SK_ID_CURR,vin_avg_rejections_last_12_months
0,100006,1
1,100027,1
2,100035,8
3,100036,3
4,100043,5
...,...,...
54071,456228,1
54072,456231,3
54073,456234,1
54074,456243,2


In [33]:
# # 23. Vintage - Total Loan Amount by Contract Type (based on AMT_CREDIT and NAME_CONTRACT_TYPE)
# def vin_total_loan_amount_by_contract_type(previous_application, agg_funcs=aggregators):
#     # Aggregating loan amounts by SK_ID_CURR and NAME_CONTRACT_TYPE
#     total_loan = previous_application.groupby(['SK_ID_CURR', 'NAME_CONTRACT_TYPE'])['AMT_CREDIT'].agg(agg_funcs).reset_index()
#     # Flattening the column names for aggregation results
#     total_loan.columns = ['SK_ID_CURR', 'NAME_CONTRACT_TYPE'] + [f'AMT_CREDIT_{agg_func}' for agg_func in agg_funcs]
#     # Pivoting the table to make NAME_CONTRACT_TYPE as columns
#     total_loan_pivot = total_loan.pivot(index='SK_ID_CURR', columns='NAME_CONTRACT_TYPE').reset_index()
#     # Flattening the MultiIndex columns
#     total_loan_pivot.columns = [f'vin_total_loan_amount_{col[1].lower()}_{col[0].split("_")[-1]}' if col[1] else col[0] for col in total_loan_pivot.columns]

#     return total_loan_pivot

# # Sample usage
# vin_total_loan_amount_by_contract_type(previous_application)

In [34]:
# # 24. Vintage - Average Annuity to Credit Ratio (based on AMT_ANNUITY and AMT_CREDIT)
# def vin_avg_annuity_credit_ratio(application_train):
#     application_train['vin_avg_annuity_credit_ratio'] = application_train['AMT_ANNUITY'] / application_train['AMT_CREDIT']
#     avg_ratio = application_train[['SK_ID_CURR', 'vin_avg_annuity_credit_ratio']].copy()
#     avg_ratio = avg_ratio.replace({'vin_avg_annuity_credit_ratio': {np.inf: np.nan, -np.inf: np.nan}})
#     return avg_ratio

# vin_avg_annuity_credit_ratio(application_train)

In [35]:
# 25. Vintage - Days Since Last Document Update (based on DAYS_ID_PUBLISH)
def vin_days_since_last_document_update(application_train, agg_funcs=['max']):
    last_doc_update = vin_generic_aggregator(application_train, 'SK_ID_CURR', 'DAYS_ID_PUBLISH', agg_funcs)
    for agg_func in agg_funcs:
        last_doc_update[f'vin_days_since_last_document_update_{agg_func}'] = last_doc_update[f'DAYS_ID_PUBLISH_{agg_func}'] * -1
    return last_doc_update[['SK_ID_CURR'] + [f'vin_days_since_last_document_update_{agg_func}' for agg_func in agg_funcs]]

vin_days_since_last_document_update(application_train)

,SK_ID_CURR,vin_days_since_last_document_update_max
0,100002,2120
1,100003,291
2,100004,2531
3,100006,2437
4,100007,3458
...,...,...
307506,456251,1982
307507,456252,4090
307508,456253,5150
307509,456254,931


In [36]:
# 26. Vintage - Maximum Debt Amount Over Time (based on AMT_CREDIT_SUM_DEBT)
def vin_max_debt_amount(bureau, agg_funcs=['max']):
    max_debt = vin_generic_aggregator(bureau, 'SK_ID_CURR', 'AMT_CREDIT_SUM_DEBT', agg_funcs)
    for agg_func in agg_funcs:
        max_debt[f'vin_max_debt_amount_{agg_func}'] = max_debt[f'AMT_CREDIT_SUM_DEBT_{agg_func}']
    return max_debt[['SK_ID_CURR'] + [f'vin_max_debt_amount_{agg_func}' for agg_func in agg_funcs]]

vin_max_debt_amount(bureau)

,SK_ID_CURR,vin_max_debt_amount_max
0,100001,373239.00
1,100002,245781.00
2,100003,0.00
3,100004,0.00
4,100005,543087.00
...,...,...
305806,456249,163071.00
305807,456250,1840308.48
305808,456253,1624797.00
305809,456254,0.00


In [37]:
# List of functions to calculate vintage variables
vintage_functions = [
    vin_months_since_last_approved,
    vin_days_since_last_rejection,
    vin_days_since_last_default,
    vin_num_previous_applications_last_n_months,
    vin_last_loan_taken_by_category,
    vin_days_since_last_loan_closure,
    vin_days_since_first_loan_taken,
    vin_num_times_delinquent_last_n_months,
    # vin_aggregated_credit_amounts,
    vin_avg_time_between_loans,
    # vin_ratio_approved_to_rejected,
    # vin_max_credit_limit,
    # vin_num_active_loans,
    vin_loan_prolongation_frequency,
    # vin_avg_installment_income_ratio,
    vin_days_since_credit_update,
    vin_num_times_credit_limit_reached,
    # vin_credit_utilization_ratio,
    # vin_total_loan_amount_by_contract_type,
    # vin_avg_annuity_credit_ratio,
    vin_days_since_last_document_update,
    vin_max_debt_amount,

    # vin_avg_rejections_last_n_months
]

# # Generate vintage variables and merge into a single DataFrame
# resultant_df = application_train[['SK_ID_CURR']]

# for months in time_periods:
#     # Functions that require time periods
#     time_based_functions = [
#         vin_num_loans_approved_last_n_months,
#         vin_num_previous_applications_last_n_months,
#         vin_num_times_delinquent_last_n_months,
#         vin_num_overdue_loans_last_n_months,
#         vin_total_overdue_amount_last_n_months,
#         vin_avg_rejections_last_n_months

#     ]
#     for func in time_based_functions:
#         feature_df = func(previous_application if 'previous_application' in func.__code__.co_varnames else bureau, months=months)
#         resultant_df = resultant_df.merge(feature_df, on='SK_ID_CURR', how='left')

# # Adding non-time-based features
# for func in vintage_functions:
#     try:
#       feature_df = func(previous_application if 'previous_application' in func.__code__.co_varnames else bureau)
#     except:
#       feature_df = func(application_train)
#     resultant_df = resultant_df.merge(feature_df, on='SK_ID_CURR', how='left')

# # Fill rate check for each column in resultant_df
# fill_rate = resultant_df.notna().mean().reset_index()
# fill_rate.columns = ['Feature', 'FillRate']
# print(fill_rate)


In [38]:
# Create dictionaries to store feature DataFrames for each source separately
application_features = {}
previous_application_features = {}
bureau_features = {}

# Functions to generate time-period-based features
for months in time_periods:
    time_based_functions = [
        vin_num_loans_approved_last_n_months,
        vin_num_previous_applications_last_n_months,
        vin_num_times_delinquent_last_n_months,
        vin_num_overdue_loans_last_n_months,
        vin_total_overdue_amount_last_n_months,
        vin_avg_rejections_last_n_months
    ]

    for func in time_based_functions:
        # Determine which data source to use
        if 'previous_application' in func.__code__.co_varnames:
            data_source = previous_application
            storage_dict = previous_application_features
        elif 'bureau' in func.__code__.co_varnames:
            data_source = bureau
            storage_dict = bureau_features
        else:
            data_source = application_train
            storage_dict = application_features

        # Generate feature DataFrame
        feature_df = func(data_source, months=months)

        # Store the generated DataFrame in the appropriate dictionary
        feature_name = f"{func.__name__}_{months}months"
        storage_dict[feature_name] = feature_df

# Functions to generate non-time-period-based features
for func in vintage_functions:
    # Determine which data source to use
    if 'previous_application' in func.__code__.co_varnames:
        data_source = previous_application
        storage_dict = previous_application_features
    elif 'bureau' in func.__code__.co_varnames:
        data_source = bureau
        storage_dict = bureau_features
    else:
        data_source = application_train
        storage_dict = application_features

    # Generate feature DataFrame
    feature_df = func(data_source)

    # Store the generated DataFrame in the appropriate dictionary
    feature_name = f"{func.__name__}"
    storage_dict[feature_name] = feature_df

# Merge the feature DataFrames into their respective original dataframes
# This allows you to have three separate dataframes: `application_train_final`, `previous_application_final`, `bureau_final`

# Start by copying the base dataframes
application_train_final = application_train[['SK_ID_CURR']]
previous_application_final = previous_application[['SK_ID_CURR']]
bureau_final = bureau[['SK_ID_CURR']]

# Merge each DataFrame from the corresponding dictionary
for feature_name, feature_df in application_features.items():
    application_train_final = application_train_final.merge(feature_df, on='SK_ID_CURR', how='left')

for feature_name, feature_df in previous_application_features.items():
    previous_application_final = previous_application_final.merge(feature_df, on='SK_ID_CURR', how='left')

for feature_name, feature_df in bureau_features.items():
    bureau_final = bureau_final.merge(feature_df, on='SK_ID_CURR', how='left')

# Fill rate check for each column in each dataframe
def print_fill_rate(df, df_name):
    fill_rate = df.notna().mean().reset_index()
    fill_rate.columns = ['Feature', 'FillRate']
    print(f"Fill Rate for {df_name}:")
    print(fill_rate)

print_fill_rate(application_train_final, 'Application Train')
print_fill_rate(previous_application_final, 'Previous Application')
print_fill_rate(bureau_final, 'Bureau')


Fill Rate for Application Train:
                                   Feature  FillRate
0                               SK_ID_CURR       1.0
1  vin_days_since_last_document_update_max       1.0
Fill Rate for Previous Application:
                                       Feature  FillRate
0                                   SK_ID_CURR  1.000000
1         vin_num_loans_approved_last_3_months  0.084548
2      num_previous_applications_last_3_months  0.220926
3             vin_avg_rejections_last_3_months  0.042115
4         vin_num_loans_approved_last_6_months  0.204006
5      num_previous_applications_last_6_months  0.427582
6             vin_avg_rejections_last_6_months  0.130239
7        vin_num_loans_approved_last_12_months  0.501960
8   num_previous_applications_last_12_months_x  0.729680
9            vin_avg_rejections_last_12_months  0.307329
10       vin_num_loans_approved_last_24_months  0.824828
11    num_previous_applications_last_24_months  0.906820
12           vin_avg_rejections

In [60]:
feature_df.head()

,SK_ID_CURR,vin_max_debt_amount_max
0,100001,373239.0
1,100002,245781.0
2,100003,0.0
3,100004,0.0
4,100005,543087.0


In [39]:
application_train_final.head()

,SK_ID_CURR,vin_days_since_last_document_update_max
0,100002,2120
1,100003,291
2,100004,2531
3,100006,2437
4,100007,3458


In [40]:
application_train_final = application_train_final.merge(
    application_train[['SK_ID_CURR', 'TARGET']],
    on='SK_ID_CURR',
    how='left'
)
application_train_final

,SK_ID_CURR,vin_days_since_last_document_update_max,TARGET
0,100002,2120,1
1,100003,291,0
2,100004,2531,0
3,100006,2437,0
4,100007,3458,0
...,...,...,...
307506,456251,1982,0
307507,456252,4090,0
307508,456253,5150,0
307509,456254,931,1


(307511, 3)

In [ ]:
application_train_final.to_csv('/content/drive/MyDrive/UMD/DATA602_DATA_Science/Project/Data Segregator /SavedData/finalized_application_train.csv', index=False)

In [43]:
previous_application_final.head()

,SK_ID_CURR,vin_num_loans_approved_last_3_months,num_previous_applications_last_3_months,vin_avg_rejections_last_3_months,vin_num_loans_approved_last_6_months,num_previous_applications_last_6_months,vin_avg_rejections_last_6_months,vin_num_loans_approved_last_12_months,num_previous_applications_last_12_months_x,vin_avg_rejections_last_12_months,...,num_previous_applications_last_36_months,vin_avg_rejections_last_36_months,vin_months_since_last_approved_max,days_since_last_rejection_max,num_previous_applications_last_12_months_y,vin_last_loan_cash loans_loan,vin_last_loan_consumer loans_loan,vin_last_loan_revolving loans_loan,vin_last_loan_xna_loan,vin_avg_time_between_loans
0,271877,1.0,1.0,NaN,1.0,1.0,NaN,1.0,1.0,NaN,...,3.0,1.0,2.433333,472.0,1.0,NaN,73.0,NaN,NaN,475.000000
1,108129,NaN,NaN,NaN,1.0,1.0,NaN,1.0,1.0,NaN,...,5.0,NaN,5.466667,NaN,1.0,164.0,619.0,957.0,NaN,208.800000
2,122040,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,NaN,...,2.0,NaN,10.033333,NaN,2.0,301.0,2576.0,2405.0,NaN,1137.500000
3,176158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0,...,8.0,3.0,17.066667,218.0,3.0,218.0,2640.0,NaN,NaN,435.400000
4,202054,NaN,7.0,4.0,NaN,7.0,4.0,4.0,16.0,8.0,...,21.0,13.0,6.300000,7.0,16.0,7.0,189.0,300.0,NaN,236.571429


In [50]:
previous_application_final = previous_application_final.drop_duplicates()

In [51]:
previous_application_final.shape

(338857, 24)

In [52]:
previous_application_final = previous_application_final.merge(
    application_train[['SK_ID_CURR', 'TARGET']],
    on='SK_ID_CURR',
    how='inner'
)
previous_application_final

,SK_ID_CURR,vin_num_loans_approved_last_3_months,num_previous_applications_last_3_months,vin_avg_rejections_last_3_months,vin_num_loans_approved_last_6_months,num_previous_applications_last_6_months,vin_avg_rejections_last_6_months,vin_num_loans_approved_last_12_months,num_previous_applications_last_12_months_x,vin_avg_rejections_last_12_months,...,vin_avg_rejections_last_36_months,vin_months_since_last_approved_max,days_since_last_rejection_max,num_previous_applications_last_12_months_y,vin_last_loan_cash loans_loan,vin_last_loan_consumer loans_loan,vin_last_loan_revolving loans_loan,vin_last_loan_xna_loan,vin_avg_time_between_loans,TARGET
0,271877,1.0,1.0,NaN,1.0,1.0,NaN,1.0,1.0,NaN,...,1.0,2.433333,472.0,1.0,NaN,73.0,NaN,NaN,475.000000,0
1,108129,NaN,NaN,NaN,1.0,1.0,NaN,1.0,1.0,NaN,...,NaN,5.466667,NaN,1.0,164.0,619.0,957.0,NaN,208.800000,0
2,122040,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,NaN,...,NaN,10.033333,NaN,2.0,301.0,2576.0,2405.0,NaN,1137.500000,0
3,176158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0,...,3.0,17.066667,218.0,3.0,218.0,2640.0,NaN,NaN,435.400000,0
4,202054,NaN,7.0,4.0,NaN,7.0,4.0,4.0,16.0,8.0,...,13.0,6.300000,7.0,16.0,7.0,189.0,300.0,NaN,236.571429,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291052,277978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,27.733333,NaN,NaN,NaN,832.0,NaN,NaN,NaN,0
291053,227766,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,16.500000,NaN,NaN,NaN,495.0,NaN,NaN,NaN,0
291054,239799,NaN,NaN,NaN,1.0,1.0,NaN,1.0,1.0,NaN,...,NaN,5.333333,NaN,1.0,NaN,160.0,NaN,NaN,NaN,0
291055,353284,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,18.666667,NaN,NaN,NaN,560.0,NaN,NaN,NaN,0


In [ ]:
previous_application_final.to_csv('/content/drive/MyDrive/UMD/DATA602_DATA_Science/Project/Data Segregator /SavedData/finalized_previous_application.csv', index=False)

In [54]:
bureau_final = bureau_final.drop_duplicates()

In [55]:
bureau_final.shape

(305811, 24)

In [56]:
bureau_final.head()

,SK_ID_CURR,vin_num_times_delinquent_last_3_months,vin_num_overdue_loans_last_3_months,vin_total_overdue_amount_last_3_months,vin_num_times_delinquent_last_6_months_x,vin_num_overdue_loans_last_6_months,vin_total_overdue_amount_last_6_months,vin_num_times_delinquent_last_12_months,vin_num_overdue_loans_last_12_months,vin_total_overdue_amount_last_12_months,...,vin_num_overdue_loans_last_36_months,vin_total_overdue_amount_last_36_months,days_since_last_default_min,vin_days_since_last_loan_closure_max,vin_days_since_first_loan_taken_max,vin_num_times_delinquent_last_6_months_y,vin_loan_prolongation_frequency,vin_days_since_credit_update_min,vin_num_times_credit_limit_reached,vin_max_debt_amount_max
0,215354,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,...,NaN,0.0,NaN,153.0,43,NaN,0,1336,3.0,171342.0
7,162297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,0.0,NaN,825.0,277,NaN,0,1710,5.0,0.0
13,402440,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,...,NaN,0.0,NaN,NaN,96,NaN,0,22,1.0,76905.0
14,238881,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,...,NaN,0.0,NaN,77.0,95,NaN,0,2601,4.0,520920.0
21,222183,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,0.0,NaN,343.0,315,NaN,0,2559,3.0,1185493.5


In [57]:
bureau_final = bureau_final.merge(
    application_train[['SK_ID_CURR', 'TARGET']],
    on='SK_ID_CURR',
    how='inner'
)
bureau_final

,SK_ID_CURR,vin_num_times_delinquent_last_3_months,vin_num_overdue_loans_last_3_months,vin_total_overdue_amount_last_3_months,vin_num_times_delinquent_last_6_months_x,vin_num_overdue_loans_last_6_months,vin_total_overdue_amount_last_6_months,vin_num_times_delinquent_last_12_months,vin_num_overdue_loans_last_12_months,vin_total_overdue_amount_last_12_months,...,vin_total_overdue_amount_last_36_months,days_since_last_default_min,vin_days_since_last_loan_closure_max,vin_days_since_first_loan_taken_max,vin_num_times_delinquent_last_6_months_y,vin_loan_prolongation_frequency,vin_days_since_credit_update_min,vin_num_times_credit_limit_reached,vin_max_debt_amount_max,TARGET
0,215354,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,...,0.0,NaN,153.0,43,NaN,0,1336,3.0,171342.0,0
1,162297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,0.0,NaN,825.0,277,NaN,0,1710,5.0,0.0,0
2,402440,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,...,0.0,NaN,NaN,96,NaN,0,22,1.0,76905.0,0
3,238881,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,...,0.0,NaN,77.0,95,NaN,0,2601,4.0,520920.0,0
4,222183,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,0.0,NaN,343.0,315,NaN,0,2559,3.0,1185493.5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263486,207190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,227.0,532,NaN,0,31,1.0,0.0,0
263487,324956,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,199.0,381,NaN,0,199,1.0,0.0,0
263488,448157,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1441,NaN,0,2,1.0,1076809.5,0
263489,345866,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,19.0,375,NaN,0,4,NaN,NaN,0


In [ ]:
bureau_final.to_csv('/content/drive/MyDrive/UMD/DATA602_DATA_Science/Project/Data Segregator /SavedData/finalized_bureau.csv', index=False)

In [ ]:
# Function to print count and percentage analysis of TARGET, including NaN values
def print_target_analysis(df, df_name):
    target_counts = df['TARGET'].value_counts(dropna=False)
    target_percentages = df['TARGET'].value_counts(normalize=True, dropna=False) * 100

    print(f"Target Analysis for {df_name}:")
    print("Counts (including NaNs):")
    print(target_counts)
    print("Percentages (including NaNs):")
    print(target_percentages)
    print()

# Perform analysis for `application_train_final`
print_target_analysis(application_train_final, 'application_train_final')

# Perform analysis for `previous_application_final`
print_target_analysis(previous_application_final, 'previous_application_final')

# Perform analysis for `bureau_final`
print_target_analysis(bureau_final, 'bureau_final')


Target Analysis for application_train_final:
Counts (including NaNs):
TARGET
0    282686
1     24825
Name: count, dtype: int64
Percentages (including NaNs):
TARGET
0    91.927118
1     8.072882
Name: proportion, dtype: float64

Target Analysis for previous_application_final:
Counts (including NaNs):
TARGET
0.0    1291341
NaN     256513
1.0     122360
Name: count, dtype: int64
Percentages (including NaNs):
TARGET
0.0    77.315901
NaN    15.358092
1.0     7.326007
Name: proportion, dtype: float64

Target Analysis for bureau_final:
Counts (including NaNs):
TARGET
0.0    1350805
NaN     251103
1.0     114520
Name: count, dtype: int64
Percentages (including NaNs):
TARGET
0.0    78.698611
NaN    14.629393
1.0     6.671996
Name: proportion, dtype: float64

